In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
import itertools

Using TensorFlow backend.


In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/test'
nb_train_samples = 2048
nb_validation_samples = 512

generator_seed = 0

In [4]:
def get_model_with_input_shape(img_width, img_height):
    if K.image_data_format() == 'channels_first':
        input_shape = (3, img_width, img_height)
    else:
        input_shape = (img_width, img_height, 3)

    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    
    return model

In [5]:
def get_generators(img_width, img_height, batch_size, rotation_range, zoom_range):
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        shear_range=0.2,
        zoom_range=zoom_range,
        rotation_range=rotation_range,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(
        rescale=1. / 255,
        zoom_range=zoom_range,
        rotation_range=rotation_range)
    
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        seed=generator_seed)

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary',
        seed=generator_seed)
    
    return (train_generator, validation_generator)

In [6]:
def train_model(model, train_generator, validation_generator, batch_size, epochs):
    model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

In [7]:
def evaluate_model(model, validation_generator, batch_size):
    score = model.evaluate_generator(validation_generator, steps=nb_validation_samples // batch_size, verbose=1)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    return score

In [8]:
image_sizes = [256, 512]
batch_sizes = [16, 32]
epochs = [4, 8]
rotation_ranges = [40, 50]
zoom_ranges = [0.20, 0.30, 0.40]

In [9]:
results = dict()
combinations = [image_sizes, batch_sizes, epochs, rotation_ranges, zoom_ranges]
for (image_size, batch_size, epochs, rotation_range, zoom_range) in itertools.product(*combinations):
    print("Trying model with image_size:" + str(image_size) + ", batch_size:" + str(batch_size) + ", epochs:" + str(epochs) + ", rotation_range:" + str(rotation_range) + ", zoom_range:" + str(zoom_range))
    
    model = get_model_with_input_shape(image_size, image_size)
    train_generator, validation_generator = get_generators(image_size, image_size, batch_size, rotation_range, zoom_range)
    train_model(model, train_generator, validation_generator, batch_size, epochs)
    print("---")
    results[(image_size, batch_size, epochs, rotation_range, zoom_range)] = evaluate_model(model, validation_generator, batch_size)
    print("-----")

Trying model with image_size:256, batch_size:16, epochs:4, rotation_range:40, zoom_range:0.2
Found 46 images belonging to 2 classes.
Found 12 images belonging to 2 classes.
Epoch 1/4
128/128 [==============================] - 164s 1s/step - loss: 1.6136 - acc: 0.7109 - val_loss: 0.6317 - val_acc: 0.6589
Epoch 2/4
128/128 [==============================] - 142s 1s/step - loss: 0.4499 - acc: 0.7681 - val_loss: 1.1285 - val_acc: 0.7448
Epoch 3/4
128/128 [==============================] - 142s 1s/step - loss: 0.2652 - acc: 0.8927 - val_loss: 2.0740 - val_acc: 0.7734
Epoch 4/4
128/128 [==============================] - 142s 1s/step - loss: 0.2018 - acc: 0.9238 - val_loss: 3.6694 - val_acc: 0.7057
---
32/32 [==============================] - 25s 790ms/step
Test loss: 3.519182190299034
Test accuracy: 0.7135416753590107
-----
Trying model with image_size:256, batch_size:16, epochs:4, rotation_range:40, zoom_range:0.3
Found 46 images belonging to 2 classes.
Found 12 images belonging to 2 classe

128/128 [==============================] - 143s 1s/step - loss: 0.2594 - acc: 0.8981 - val_loss: 1.4357 - val_acc: 0.6380
Epoch 4/8
128/128 [==============================] - 143s 1s/step - loss: 0.1801 - acc: 0.9362 - val_loss: 1.9474 - val_acc: 0.6250
Epoch 5/8
128/128 [==============================] - 144s 1s/step - loss: 0.1265 - acc: 0.9632 - val_loss: 2.0910 - val_acc: 0.6979
Epoch 6/8
128/128 [==============================] - 143s 1s/step - loss: 0.1211 - acc: 0.9577 - val_loss: 2.8689 - val_acc: 0.6250
Epoch 7/8
128/128 [==============================] - 143s 1s/step - loss: 0.1293 - acc: 0.9641 - val_loss: 3.0476 - val_acc: 0.6380
Epoch 8/8
128/128 [==============================] - 144s 1s/step - loss: 0.1653 - acc: 0.9706 - val_loss: 2.7943 - val_acc: 0.5729
---
32/32 [==============================] - 25s 794ms/step
Test loss: 2.585416592657566
Test accuracy: 0.5624999916180968
-----
Trying model with image_size:256, batch_size:16, epochs:8, rotation_range:50, zoom_range:

16/16 [==============================] - 13s 801ms/step
Test loss: 0.5364042222499847
Test accuracy: 0.6145833283662796
-----
Trying model with image_size:256, batch_size:32, epochs:4, rotation_range:50, zoom_range:0.3
Found 46 images belonging to 2 classes.
Found 12 images belonging to 2 classes.
Epoch 1/4
64/64 [==============================] - 106s 2s/step - loss: 1.3445 - acc: 0.6990 - val_loss: 0.5724 - val_acc: 0.6667
Epoch 2/4
64/64 [==============================] - 102s 2s/step - loss: 0.5966 - acc: 0.7155 - val_loss: 0.4869 - val_acc: 0.7344
Epoch 3/4
64/64 [==============================] - 101s 2s/step - loss: 0.5061 - acc: 0.7467 - val_loss: 0.5338 - val_acc: 0.6354
Epoch 4/4
64/64 [==============================] - 102s 2s/step - loss: 0.4829 - acc: 0.7629 - val_loss: 0.5101 - val_acc: 0.6042
---
16/16 [==============================] - 13s 802ms/step
Test loss: 0.5065992418676615
Test accuracy: 0.5781249925494194
-----
Trying model with image_size:256, batch_size:32, ep

Epoch 8/8
64/64 [==============================] - 104s 2s/step - loss: 0.1492 - acc: 0.9508 - val_loss: 2.5948 - val_acc: 0.5417
---
16/16 [==============================] - 13s 810ms/step
Test loss: 2.2509490847587585
Test accuracy: 0.5572916567325592
-----
Trying model with image_size:256, batch_size:32, epochs:8, rotation_range:50, zoom_range:0.4
Found 46 images belonging to 2 classes.
Found 12 images belonging to 2 classes.
Epoch 1/8
64/64 [==============================] - 107s 2s/step - loss: 1.6392 - acc: 0.6793 - val_loss: 0.6034 - val_acc: 0.6667
Epoch 2/8
64/64 [==============================] - 102s 2s/step - loss: 0.6259 - acc: 0.7164 - val_loss: 0.5131 - val_acc: 0.7240
Epoch 3/8
64/64 [==============================] - 103s 2s/step - loss: 0.4895 - acc: 0.7539 - val_loss: 1.0549 - val_acc: 0.5260
Epoch 4/8
64/64 [==============================] - 102s 2s/step - loss: 0.4443 - acc: 0.8070 - val_loss: 1.2175 - val_acc: 0.5469
Epoch 5/8
64/64 [==============================

Epoch 2/8
128/128 [==============================] - 242s 2s/step - loss: 4.4928 - acc: 0.7182 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 3/8
128/128 [==============================] - 242s 2s/step - loss: 4.5349 - acc: 0.7155 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 4/8
128/128 [==============================] - 260s 2s/step - loss: 4.5238 - acc: 0.7162 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 5/8
128/128 [==============================] - 243s 2s/step - loss: 4.4837 - acc: 0.7188 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 6/8
128/128 [==============================] - 243s 2s/step - loss: 4.5172 - acc: 0.7167 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 7/8
128/128 [==============================] - 244s 2s/step - loss: 4.5082 - acc: 0.7172 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 8/8
128/128 [==============================] - 243s 2s/step - loss: 4.5137 - acc: 0.7169 - val_loss: 5.3141 - val_acc: 0.6667
---
32/32 [==============================] - 40s 1s/step
Test loss: 5.314128

Found 12 images belonging to 2 classes.
Epoch 1/4
64/64 [==============================] - 186s 3s/step - loss: 0.8464 - acc: 0.6767 - val_loss: 0.5496 - val_acc: 0.6667
Epoch 2/4
64/64 [==============================] - 179s 3s/step - loss: 0.5736 - acc: 0.7357 - val_loss: 0.8015 - val_acc: 0.5885
Epoch 3/4
64/64 [==============================] - 179s 3s/step - loss: 0.4319 - acc: 0.7959 - val_loss: 0.9781 - val_acc: 0.4427
Epoch 4/4
64/64 [==============================] - 177s 3s/step - loss: 0.3653 - acc: 0.8415 - val_loss: 0.8237 - val_acc: 0.5833
---
16/16 [==============================] - 21s 1s/step
Test loss: 0.8240719884634018
Test accuracy: 0.5937499962747097
-----
Trying model with image_size:512, batch_size:32, epochs:4, rotation_range:50, zoom_range:0.2
Found 46 images belonging to 2 classes.
Found 12 images belonging to 2 classes.
Epoch 1/4
64/64 [==============================] - 188s 3s/step - loss: 0.8233 - acc: 0.6836 - val_loss: 0.8091 - val_acc: 0.6667
Epoch 2/4


64/64 [==============================] - 180s 3s/step - loss: 0.2076 - acc: 0.9184 - val_loss: 2.5690 - val_acc: 0.6250
Epoch 8/8
64/64 [==============================] - 180s 3s/step - loss: 0.1818 - acc: 0.9421 - val_loss: 2.8400 - val_acc: 0.5781
---
16/16 [==============================] - 22s 1s/step
Test loss: 2.809299886226654
Test accuracy: 0.5781249813735485
-----
Trying model with image_size:512, batch_size:32, epochs:8, rotation_range:50, zoom_range:0.3
Found 46 images belonging to 2 classes.
Found 12 images belonging to 2 classes.
Epoch 1/8
64/64 [==============================] - 188s 3s/step - loss: 4.6250 - acc: 0.7037 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 2/8
64/64 [==============================] - 179s 3s/step - loss: 4.5783 - acc: 0.7128 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 3/8
64/64 [==============================] - 189s 3s/step - loss: 4.3516 - acc: 0.7270 - val_loss: 5.3141 - val_acc: 0.6667
Epoch 4/8
64/64 [==============================] - 179s 3s/st

In [21]:
results_acc = {key: value[1] for key, value in results.items()}
highest_acc_comb = max(results_acc, key=results_acc.get)
print("Combination that produces highest test accuracy ("+str(results_acc[highest_acc_comb])+"): " + str(highest_acc_comb))

results_loss = {key: value[0] for key, value in results.items()}
lowest_loss_comb = min(results_loss, key=results_loss.get)
print("Combination that produces lowest test loss ("+str(results_loss[lowest_loss_comb])+"): " + str(lowest_loss_comb))

Combination that produces highest test accuracy (0.7343750037252903): (256, 16, 4, 40, 0.4)
Combination that produces lowest test loss (0.5065992418676615): (256, 32, 4, 50, 0.3)


In [24]:
import pickle
with open("results.pkl", "wb") as results_file:
    pickle.dump(results, results_file)


In [25]:
get_model_with_input_shape(highest_acc_comb[0], highest_acc_comb[0]).summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_148 (Conv2D)          (None, 254, 254, 32)      896       
_________________________________________________________________
activation_246 (Activation)  (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_148 (MaxPoolin (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 125, 125, 32)      9248      
_________________________________________________________________
activation_247 (Activation)  (None, 125, 125, 32)      0         
_________________________________________________________________
max_pooling2d_149 (MaxPoolin (None, 62, 62, 32)        0         
_________________________________________________________________
conv2d_150 (Conv2D)          (None, 60, 60, 64)        18496     
__________